### Joining Data
Joining data is understandable when you are supposed to combine two given tables.

In [3]:
from google.cloud import bigquery
import numpy as np
import pandas as pd
import os

In [4]:
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "/Users/xlyue/Documents/kaggle learning/Intro to SQL/sql-bq-244804-803e47cbe74e.json"

In [5]:
client = bigquery.Client()
dataset_ref = client.dataset('github_repos', project = 'bigquery-public-data')
# API request - fetch the dataset
dataset = client.get_dataset(dataset_ref)

In [6]:
tables = list(client.list_tables(dataset))
for table in tables:
    print(table.table_id)
print()
print(len(tables))

commits
contents
files
languages
licenses
sample_commits
sample_contents
sample_files
sample_repos

9


In [7]:
# Construct a reference to the "licenses" table
licenses_ref = dataset_ref.table('licenses')

# API request - fetch the table
licenses_table = client.get_table(licenses_ref)

# Preview the first five lines of the "licenses" table
client.list_rows(licenses_table, max_results = 5).to_dataframe()

,repo_name,license
0,azuredream/chat_server-client,artistic-2.0
1,Egyptian19/JemCraft,artistic-2.0
2,ZioRiP/cookie,artistic-2.0
3,ajs/perl6-log,artistic-2.0
4,JohanPotgieter/Internet,artistic-2.0


In [8]:
# Construct a reference to the "sample_files" table
files_ref = dataset_ref.table('sample_files')

# API request - fetch the table
files_table = client.get_table(files_ref)

# Preview the first five lines of the "sample_files" table
client.list_rows(files_table, max_results=5).to_dataframe()

,repo_name,ref,path,mode,id,symlink_target
0,git/git,refs/heads/master,RelNotes,40960,62615ffa4e97803da96aefbc798ab50f949a8db7,Documentation/RelNotes/2.10.0.txt
1,np/ling,refs/heads/master,tests/success/plug_compose.t/plug_compose.ll,40960,0c1605e4b447158085656487dc477f7670c4bac1,../../../fixtures/all/plug_compose.ll
2,np/ling,refs/heads/master,fixtures/strict-par-success/parallel_assoc_lef...,40960,b59bff84ec03d12fabd3b51a27ed7e39a180097e,../all/parallel_assoc_left.ll
3,np/ling,refs/heads/master,fixtures/sequence/parallel_assoc_2tensor2_left.ll,40960,f29523e3fb65702d99478e429eac6f801f32152b,../all/parallel_assoc_2tensor2_left.ll
4,np/ling,refs/heads/master,fixtures/success/my_dual.ll,40960,38a3af095088f90dfc956cb990e893909c3ab286,../all/my_dual.ll


Now we focus on common information in both tables.

In [9]:
query = """
           SELECT L.license, COUNT(1) AS number_of_files
           FROM `bigquery-public-data.github_repos.sample_files` AS sf
           INNER JOIN `bigquery-public-data.github_repos.licenses` AS L ON sf.repo_name = L.repo_name
           GROUP BY L.license
           ORDER BY number_of_files DESC
           """

In [10]:
safe_config = bigquery.QueryJobConfig(maximum_bytes_billed = 10**10)
query_job = client.query(query, job_config = safe_config)

# API request - run the query, and convert the results to a pandas DataFrame
file_count_by_license = query_job.to_dataframe()

In [12]:
file_count_by_license.head()

,license,number_of_files
0,mit,20624490
1,gpl-2.0,17119548
2,apache-2.0,7232387
3,gpl-3.0,4960665
4,bsd-3-clause,2944149


Exercises

In [13]:
client = bigquery.Client()

# Construct a reference to the "stackoverflow" dataset
dataset_ref = client.dataset('stackoverflow', project = 'bigquery-public-data')

# API request - fetch the dataset
dataset = client.get_dataset(dataset_ref)

In [14]:
tables = list(client.list_tables(dataset))
for table in tables:
    print(table.table_id)
print()
print(len(tables))

badges
comments
post_history
post_links
posts_answers
posts_moderator_nomination
posts_orphaned_tag_wiki
posts_privilege_wiki
posts_questions
posts_tag_wiki
posts_tag_wiki_excerpt
posts_wiki_placeholder
stackoverflow_posts
tags
users
votes

16


Review relevant tables: `posts_answers` & `posts_questions`

In [15]:
# Construct a reference to the "posts_answers" table
answers_table_ref = dataset_ref.table('posts_answers')

# API request - fetch the table
answers_table = client.get_table(answers_table_ref)

# Preview the first five lines of the "posts_answers" table
client.list_rows(answers_table, max_results = 5).to_dataframe()

,id,body,comment_count,community_owned_date,creation_date,last_activity_date,last_edit_date,last_editor_display_name,last_editor_user_id,owner_display_name,owner_user_id,parent_id,post_type_id,score,tags
0,47949879,<p>Building onto what you have nearly got..</p...,0,None,2017-12-23 04:57:17.177000+00:00,2017-12-23 04:57:17.177000+00:00,NaT,,NaN,,5602957,47949810,2,2,
1,15503802,"<p>Normally, ""naturally aligned"" means that an...",1,None,2013-03-19 15:34:54.153000+00:00,2013-03-19 15:34:54.153000+00:00,NaT,,NaN,,179910,15503537,2,3,
2,3497784,"<p>Looking with Reflector, it seems to indicat...",0,None,2010-08-16 22:04:25.970000+00:00,2010-08-16 22:04:25.970000+00:00,NaT,,NaN,,1831,3497732,2,2,
3,39122150,<p>Probably not. I don't know what version of ...,1,None,2016-08-24 11:30:55.667000+00:00,2016-08-24 11:30:55.667000+00:00,2017-05-23 12:09:42.380000+00:00,,-1.0,,2868335,39110336,2,3,
4,36403800,"<p><code>replace(',', '')</code> only replaces...",0,None,2016-04-04 13:18:15.710000+00:00,2016-04-04 13:18:15.710000+00:00,NaT,,NaN,,2825245,36403626,2,2,


In [16]:
# Construct a reference to the "posts_questions" table
questions_table_ref = dataset_ref.table('posts_questions')

# API request - fetch the table
questions_table = client.get_table(questions_table_ref)

# Preview the first five lines of the "posts_questions" table
client.list_rows(questions_table, max_results = 5).to_dataframe()

,id,title,body,accepted_answer_id,answer_count,comment_count,community_owned_date,creation_date,favorite_count,last_activity_date,last_edit_date,last_editor_display_name,last_editor_user_id,owner_display_name,owner_user_id,post_type_id,score,tags,view_count
0,9804136,"How do I clear an array of a structure in C, a...",<p>Two questions:</p>\n\n<ol>\n<li>How to quic...,9804289.0,3,2,None,2012-03-21 11:54:26.897000+00:00,NaN,2012-03-21 14:33:12.307000+00:00,2012-03-21 14:33:12.307000+00:00,,882600.0,,662586,1,1,c|struct,6462
1,44896935,How to implement threading to run two bash she...,<p>I have to record a wav file and at the same...,44897223.0,2,6,None,2017-07-04 04:14:50.020000+00:00,1.0,2017-07-11 09:23:08.210000+00:00,2017-07-11 09:23:08.210000+00:00,,3178797.0,,3178797,1,1,python|multithreading|bash|shell|python-multit...,411
2,3059091,How to remove carriage returns from output of ...,<p>I am using wordpress as a CMS and trying to...,3592816.0,7,5,None,2010-06-17 05:26:30.147000+00:00,11.0,2017-05-11 18:05:49.817000+00:00,2010-08-28 06:13:26.817000+00:00,,82330.0,,82330,1,26,php|javascript|wordpress|google-maps,73286
3,8926063,"Code Coverage: Why is end marker red (End If, ...",<p>I use MS-Test with Visual Studio 2010 and V...,8934375.0,4,0,None,2012-01-19 12:32:19.227000+00:00,NaN,2012-01-19 22:33:52.037000+00:00,2012-01-19 13:16:56.100000+00:00,,254041.0,,254041,1,4,vb.net|visual-studio|mstest|code-coverage,566
4,52977342,incorrect checksum for freed object - object w...,<p>I was hoping the return values in c++11 are...,NaN,0,10,None,2018-10-24 20:17:45.643000+00:00,1.0,2018-10-24 20:17:45.643000+00:00,NaT,,NaN,,1935611,1,0,c++|c++11,203


We need `parent_id` from `posts_answers`

and we also find that both tables have column `owner_user_id`

Try with `%` and `Like`

Find rows that contain keywords `bigquery`

In [17]:
questions_query = """
                          SELECT id, title, owner_user_id
                          FROM `bigquery-public-data.stackoverflow.posts_questions`
                          WHERE tags like '%bigquery%'
                          """

In [18]:
safe_config = bigquery.QueryJobConfig(maximum_bytes_billed = 10**10)
questions_query_job = client.query(questions_query, job_config = safe_config)

In [19]:
# API request - run the query, and return a pandas DataFrame
questions_results = questions_query_job.to_dataframe()

print(questions_results.head())

         id                                              title  owner_user_id
0  25139022  Copy multiple BigQuery tables using BigQuery J...      1311879.0
1  33062382  Bigquery Streaming inserts, persistent or new ...       367985.0
2  36319549  Insert data to nested fields of a table at Goo...      3077252.0
3  41624894  Is it possible to write a windowed unbounded P...      1267390.0
4  52623870         Google Data Studio (BigQuery) from 1 to 01            NaN


Try with `join`

In [20]:
answers_query = """
                        SELECT a.id, a.body, a.owner_user_id
                        FROM `bigquery-public-data.stackoverflow.posts_questions` as q
                        inner join `bigquery-public-data.stackoverflow.posts_answers` as a on q.id = a.parent_id
                        WHERE q.tags like '%bigquery%'
                        """

In [23]:
safe_config = bigquery.QueryJobConfig(maximum_bytes_billed = 10**11)
answers_query_job = client.query(answers_query, job_config = safe_config)

In [24]:
# API request - run the query, and return a pandas DataFrame
answers_results = answers_query_job.to_dataframe()

answers_results.head()

,id,body,owner_user_id
0,22483870,<p>The way Google Cloud tools are distributed ...,2840536.0
1,51849188,<p>You can start with visiting bigquery site i...,9367402.0
2,55244826,<p>I think I figured a way to do it but I am n...,8229534.0
3,17125636,<p>I need to look into your code. Please post ...,1798394.0
4,55259543,"<p>As far as I understand the query, you want ...",9384667.0


In [25]:
bigquery_experts_query = """
                                     select a.owner_user_id as user_id, count(1) as number_of_answers
                                     from `bigquery-public-data.stackoverflow.posts_questions` as q
                                     inner join `bigquery-public-data.stackoverflow.posts_answers` as a on q.id = a.parent_id
                                     where q.tags like '%bigquery%'
                                     group by user_id
                                     """

In [26]:
safe_config = bigquery.QueryJobConfig(maximum_bytes_billed = 10**10)
bigquery_experts_query_job = client.query(bigquery_experts_query, job_config = safe_config)

In [27]:
# API request - run the query, and return a pandas DataFrame
bigquery_experts_results = bigquery_experts_query_job.to_dataframe()

bigquery_experts_results.head()

,user_id,number_of_answers
0,1144035.0,442
1,1451653.0,1
2,9959433.0,2
3,6823528.0,25
4,8260589.0,4


If we would like to convert what we did above into a function service for our website:

In [28]:
def expert_finder(topic, client):
    '''
    Inputs:
        topic: A string with the topic of interest
        client: A Client object that specifies the connection to the Stack Overflow dataset

    Outputs:
        results: A DataFrame with columns for user_id and number_of_answers.
    '''
    my_query = """
                     SELECT a.owner_user_id AS user_id, COUNT(1) AS number_of_answers
                     FROM `bigquery-public-data.stackoverflow.posts_questions` AS q
                     INNER JOIN `bigquery-public-data.stackoverflow.posts_answers` AS a ON q.id = a.parent_Id
                     WHERE q.tags like '%' + topic + '%'
                     GROUP BY user_id
                     """
    
    safe_config = bigquery.QueryJobConfig(maximum_bytes_billed = 10**9)      
    my_query_job = client.query(my_query, job_config = safe_config)

    # API request - run the query, and return a pandas DataFrame
    results = my_query_job.to_dataframe()

    return results